## Trabajar con datos JSON en Amazon DynamoDB

Amazon DynamoDB le permite almacenar objetos JSON en atributos y realizar muchas operaciones con estos objetos, incluidos el filtrado, la actualización y la eliminación. Se trata de una capacidad muy potente, ya que permite a las aplicaciones almacenar objetos (datos JSON, matrices) directamente en las tablas de DynamoDB y conservar la capacidad de utilizar atributos anidados dentro de estos objetos en sus operaciones de base de datos. En esta publicación, exploramos algunas de las cosas que puede hacer con objetos JSON en DynamoDB.

Utilizaremos una tabla sencilla de DynamoDB que rastrea información sobre pedidos de comida rápida. El siguiente código es un pedido de ejemplo:

`{ PK: 12, SK: 1, details: { entree: “burger”, sides: [ “fries”, “soda” ]}, status: “WIP” }`

Una orden completada tiene el siguiente aspecto:

`{ PK: 15, SK: 1, details: { entree: “salad”, sides: [ “apple”, “water” ]}, status: “DONE” }`

Un pedido con tres artículos tiene el siguiente aspecto:

`{ PK: 14, SK: 1, details: { entree: “salad”, sides: [ “apple”, “water” ]}, status: “WIP” }`
<br>
`{ PK: 14, SK: 2, details: { entree: “BLT sw”, sides: [ “water” ]}, status: “WIP” }`

In [1]:
import pandas as pd
from spdynamodb import DynamoTable
from time import sleep
import json
from decimal import Decimal

In [2]:
dt = DynamoTable()
try:
    dt.select_table('Orders')
    print(dt)
except:
    dt.create_table(
        table_name='Orders',
        partition_key='PK',
        partition_key_type='N',
        sort_key='SK',
        sort_key_type='N',
        provisioned=False
    )

- Table name: Orders            
- Table arn: arn:aws:dynamodb:us-east-1:089715336747:table/Orders            
- Table creation: 2023-05-18 09:47:23.073000-03:00            
- [{'AttributeName': 'PK', 'KeyType': 'HASH'}, {'AttributeName': 'SK', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'PK', 'AttributeType': 'N'}, {'AttributeName': 'SK', 'AttributeType': 'N'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


In [3]:
%%writefile orders.json
[
    {
        "PK": 12,
        "SK": 1,
        "details": {"entree": "burger", "sides": ["fries", "soda"]},
        "status": "WIP"
    },
    {
        "PK": 12,
        "SK": 2,
        "details": {"entree": "BLT sw", "sides": ["water"]},
        "status": "WIP"
    },
    {
        "PK": 14,
        "SK": 1,
        "details": {"entree": "salad", "sides": ["apple", "water"]},
        "status": "WIP"
    },
    {
        "PK": 14,
        "SK": 2,
        "details": {"entree": "BLT sw", "sides": ["water"]},
        "status": "WIP"
    },
    {
        "PK": 15,
        "SK": 1,
        "details": {"entree": "burger", "sides": ["fries", "soda"]},
        "status": "DONE"
    }
]   

Writing orders.json


In [4]:
dt.load_json("orders.json")

### Consultar los datos
Puede encontrar el primer elemento de la orden 14 ejecutando la siguiente consulta:

In [3]:
dt.query(pk_value=14, sk_value=1)

{'SK': 1,
 'details': {'entree': 'salad', 'sides': ['apple', 'water']},
 'PK': 14,
 'status': 'WIP'}

También puede realizar otras consultas, como encontrar todas las líneas de pedido que incluyen agua y que aún no se ha completado:

In [12]:
statment = "SELECT * FROM Orders WHERE status != 'DONE' and contains(details.sides, 'water')"
dt.query_partiql(query=statment, consumed_capacity="TOTAL")

ExecuteStatement executed successfully.
Consumed Capacity: 2.0


[{'SK': 2,
  'details': {'entree': 'BLT sw', 'sides': ['water']},
  'PK': 12,
  'status': 'WIP'}]

Del mismo modo, si una hamburguesa está lista, podemos determinar a qué pedido debe adjuntarse:

In [4]:
statment = "SELECT PK, SK FROM Orders WHERE status = 'WIP' and contains(details.entree, 'burger')"
dt.query_partiql(query=statment, consumed_capacity="TOTAL")

ExecuteStatement executed successfully.
Consumed Capacity: 2.0


[{'PK': 12, 'SK': 1}]

### Actualizar elementos

También puede actualizar estos pedidos; por ejemplo, el cliente que realizó el pedido 14, artículo 1, cambió de opinión y en lugar de agua desea un refresco:

In [6]:
dt.table.update_item(
    Key = {"PK": 14, "SK": 1},
    UpdateExpression = "SET details.sides[1] = :s",
    ExpressionAttributeValues = {":s": "soda"},
    ReturnValues = "UPDATED_NEW"
)

{'Attributes': {'details': {'sides': ['soda']}},
 'ResponseMetadata': {'RequestId': '4ER8QTL1JNJIK5JSR49FDS37D7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 18 May 2023 14:54:01 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '63',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4ER8QTL1JNJIK5JSR49FDS37D7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3120251123'},
  'RetryAttempts': 0}}

Observe que la actualización hace referencia a elementos de la matriz details.sides por su índice. Aquí se actualiza el segundo elemento (índice 1).

### Añadir atributos a un objeto incrustado y eliminarlos de él
Si un cliente tiene una petición especial, puede añadirla al pedido:

In [8]:
dt.table.update_item(
    Key = {"PK": 14, "SK": 1},
    UpdateExpression = "SET details.notes = :s",
    ExpressionAttributeValues = {":s": "extra dressing"},
    ReturnValues = "UPDATED_NEW"
)

{'Attributes': {'details': {'notes': 'extra dressing'}},
 'ResponseMetadata': {'RequestId': '77JDF3EJNGCUJ67K0ES25PA017VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 18 May 2023 15:11:23 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '65',
   'connection': 'keep-alive',
   'x-amzn-requestid': '77JDF3EJNGCUJ67K0ES25PA017VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1420992425'},
  'RetryAttempts': 0}}

Si el cliente que pidió el sándwich BLT decidió que no quería el agua después de todo, podemos utilizar el siguiente código:

In [11]:
dt.table.update_item(
    Key = {"PK": 14, "SK": 2},
    UpdateExpression = "REMOVE details.sides[0]",
    #ExpressionAttributeValues = {":s": "BLT sw"},
    ReturnValues = "UPDATED_NEW"
)

{'ResponseMetadata': {'RequestId': '14L3TJVLS6NK7JFMV48F2Q82S7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 18 May 2023 15:31:12 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '14L3TJVLS6NK7JFMV48F2Q82S7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}